<a href="https://colab.research.google.com/github/opticalcode/Introduction-to-natural-language-processing/blob/master/AI%E4%BD%9C%E8%A9%9E%E4%BD%9C%E6%9B%B2%E3%81%AE%E6%9B%B2%E3%80%8E%E6%99%82%E8%A8%88%E3%81%AF%E5%88%BB%E3%82%80%E3%82%88%E3%80%81%E5%83%95%E3%81%A8%E8%AA%B0%E3%81%8B%E3%82%92%E3%80%8F%E3%82%92COTOHA_API%E3%81%A7NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import urllib.request
import json
import codecs

# 「自然言語処理を簡単に扱えると噂のCOTOHA APIをPythonで使ってみた」
# https://qiita.com/gossy5454/items/83072418fb0c5f3e269f
# 上記記事のコードをベースに、Colaboratoryですぐ動くように改変。
# また、同じ処理を共通化してコード全体を短くし
# 複数のAPIのサンプルコードも追加した。
# 本改変後コードの詳細は下記参照。
# https://qiita.com/youwht/items/16e67f4ada666e679875

# COTOHA API操作用クラス
class CotohaApi:
  # 初期化
  def __init__(self, client_id, client_secret, developer_api_base_url, access_token_publish_url):
    self.client_id = client_id
    self.client_secret = client_secret
    self.developer_api_base_url = developer_api_base_url
    self.access_token_publish_url = access_token_publish_url
    self.getAccessToken()

  # アクセストークン取得
  def getAccessToken(self):
    # アクセストークン取得URL指定
    url = self.access_token_publish_url
    # ヘッダ指定
    headers={
      "Content-Type": "application/json;charset=UTF-8"
    }
    # リクエストボディ指定
    data = {
      "grantType": "client_credentials",
      "clientId": self.client_id,
      "clientSecret": self.client_secret
    }
    # リクエストボディ指定をJSONにエンコード
    data = json.dumps(data).encode()
    # リクエスト生成
    req = urllib.request.Request(url, data, headers)
    # リクエストを送信し、レスポンスを受信
    res = urllib.request.urlopen(req)
    # レスポンスボディ取得
    res_body = res.read()
    # レスポンスボディをJSONからデコード
    res_body = json.loads(res_body)
    # レスポンスボディからアクセストークンを取得
    self.access_token = res_body["access_token"]

  #各APIの共通処理
  def callCotohaApiCommon(self, url, data):
    # ヘッダ指定
    headers={
      "Authorization": "Bearer " + self.access_token,
      "Content-Type": "application/json;charset=UTF-8",
    }
    # リクエストボディ指定をJSONにエンコード
    data = json.dumps(data).encode()
    # リクエスト生成
    req = urllib.request.Request(url, data, headers)
    # リクエストを送信し、レスポンスを受信
    # 以下、リクエストエラー時のリトライ処理を入れたが、
    # 長時間放置後に再実行、などが無ければ消しても良い。
    # リトライカウントを設定
    CONNECTION_RETRY_COUNT = 2
    for counter in range(1, CONNECTION_RETRY_COUNT + 1):
      try:
        res = urllib.request.urlopen(req)
        #exceptionに行かなければループ終了
        break
      # リクエストでエラーが発生した場合の処理
      except urllib.request.HTTPError as e:
        # ステータスコードが401 Unauthorizedならアクセストークンを取得し直して再リクエスト
        if e.code == 401:
          print ("[RETRY] get access token")
          self.access_token = getAccessToken(self.client_id, self.client_secret)
          headers["Authorization"] = "Bearer " + self.access_token
          req = urllib.request.Request(url, data, headers)
        # 401以外のエラーなら原因を表示して処理自体を終了。空白を返す。
        else:
          print ("<Error> " + e.reason)
          return ""
      # それ以外のエラーは不明なので終了。
      # サーバ系のエラーの場合、だいたいURLか引数を見直すと良い
      except Exception as e:
        print(e)
        return ""
    # レスポンスボディ取得
    res_body = res.read()
    # レスポンスボディをJSONからデコード
    res_body = json.loads(res_body)
    # レスポンスボディから解析結果を取得
    return res_body

  # 構文解析API
  def callParseApi(self, sentence):
    url = self.developer_api_base_url + "nlp/v1/parse"
    data = {
        "sentence": sentence
    }
    return self.callCotohaApiCommon(url, data)

  # 固有表現抽出API
  def callNeApi(self, sentence):
    url = self.developer_api_base_url + "nlp/v1/ne"
    data = {
        "sentence": sentence
    }
    return self.callCotohaApiCommon(url, data)

  # 照応解析API
  # document は string / array(string) なので文でも、文のリストでも良い
  # ほかのdocument箇所も同様
  def callCoreferenceApi(self, document):
    url = self.developer_api_base_url + "nlp/v1/coreference"  
    data = {
      "document": document
    }
    return self.callCotohaApiCommon(url, data)

  # キーワード抽出API
  def callKeywordApi(self, document):
    url = self.developer_api_base_url + "nlp/v1/keyword"
    data = {
      "document": document
    }
    return self.callCotohaApiCommon(url, data)

  # 類似度算出API
  def callSimilarityApi(self, sentence1, sentence2):
    url = self.developer_api_base_url + "nlp/v1/similarity"
    data = {
      "s1": sentence1,
      "s2": sentence2
    }
    return self.callCotohaApiCommon(url, data)

  # 文タイプ判定API
  def callSentenceTypeApi(self, sentence):
    url = self.developer_api_base_url + "nlp/v1/sentence_type"
    data = {
      "sentence": sentence
    }
    return self.callCotohaApiCommon(url, data)

  # ユーザ属性推定API
  def callUserAttributeApi(self, document):
    url = self.developer_api_base_url + "nlp/beta/user_attribute"
    data = {
      "document": document
    }
    return self.callCotohaApiCommon(url, data)

  # 感情分析API
  def callSentimentApi(self, sentence):
    url = self.developer_api_base_url + "nlp/v1/sentiment"
    data = {
      "sentence": sentence
    }
    return self.callCotohaApiCommon(url, data)

  # 要約API
  # sentenceのサイズは、5～5000
  # sent_lenのサイズは、1～100
  def callSummaryApi(self, sentence, sent_len):
    url = self.developer_api_base_url + "nlp/beta/summary"
    data = {
      "document": sentence,
      "sent_len": sent_len
    }
    return self.callCotohaApiCommon(url, data)

  #上記以外のAPIや、各APIのオプションを追加/変更するのも同様にやればOK

if __name__ == '__main__':
  #####各自の設定を記入する箇所#####
  #登録完了直後のページに書いてあるよ
  CLIENT_ID     = "ココニアイデイカク"
  CLIENT_SECRET = "ココニシークレットカク"
  #BASE_URLは、公式ページでは、末尾にnlpを含めていないけど、
  #少し前の技術記事だと、末尾にnlpも含めている人も多いから気を付けてね
  DEVELOPER_API_BASE_URL   = "https://api.ce-cotoha.com/api/dev/"
  ACCESS_TOKEN_PUBLISH_URL = "https://api.ce-cotoha.com/v1/oauth/accesstokens"

  #####以下使用例#####
  # 各APIのコール方法や、レスポンスの解析の詳細はリファレンスを見るといいよ。
  # https://api.ce-cotoha.com/contents/reference/apireference.html

  # COTOHA APIインスタンス生成
  cotoha_api = CotohaApi(CLIENT_ID, CLIENT_SECRET, DEVELOPER_API_BASE_URL, ACCESS_TOKEN_PUBLISH_URL)

  print("■ 構文解析")
  sentence = "そう今は冷たく、君の一番を走らせてよ"
  api_result  = cotoha_api.callParseApi(sentence)
  for chunks in api_result['result']:
    print(chunks)

  print("■ 固有表現抽出")
  sentence = "最後に刻むよ、きっと飛べたら、二人は此処から、遠くまで見える"
  api_result  = cotoha_api.callNeApi(sentence)
  for chunks in api_result['result']:
    print(chunks)

  print("■ 照応解析")
  document = ["真面目なハートの傷痕","二人埋もれたね、やさしさ世界中","行かない、海は終わらないよ"]
  api_result  = cotoha_api.callCoreferenceApi(document)
  for chunks in api_result['result']['coreference']:
    print(chunks['referents'])

  print("■ キーワード抽出")
  document = ["サヨナラの涙、とにかく僕が見えている、どれだけ心が、大人になるのか目覚めていよう"]
  api_result  = cotoha_api.callKeywordApi(document)
  for chunks in api_result['result']:
    print(chunks)

  print("■ 類似度算出")
  sentence1 = "見慣れない世界へ続くよ、スタートラインを超えたら"
  sentence2 = "時計を止めてよ、僕と誰かの"
  api_result  = cotoha_api.callSimilarityApi(sentence1, sentence2)
  print(api_result['result']['score'])

  print("■ 文タイプ判定")
  sentence = "忘れず、懐かしい空に"
  api_result  = cotoha_api.callSentenceTypeApi(sentence)
  print(api_result['result']['modality'])
  #declarative（叙述）
  #interrogative（質問）
  #imperative（命令）

  print("■ ユーザ属性推定")
  document = ["言葉の退屈なんか、今も全部ここにはないよ","進み続ける時が来るから、遠くまで行くよ"]
  api_result  = cotoha_api.callUserAttributeApi(document)
  print('結果 ＝ {}'.format(api_result['result']) )

  print("■ 感情分析")
  sentence = "二人埋もれたね、やさしさ世界中、夢も消えるなら、いつか映像が見えるよ"
  api_result  = cotoha_api.callSentimentApi(sentence)
  #他のAPIの結果についても、各結果をjsonのまま見たい場合は下記のようにする。
  result_formated = json.dumps(api_result, indent=4, separators=(',', ': '))
  print (codecs.decode(result_formated, 'unicode-escape'))

  print("■ 要約")
  sentence='''タイトル：時計は刻むよ、僕と誰かを
そう今は冷たく、君の一番を走らせてよ
最後に刻むよ、きっと飛べたら
二人は此処から、遠くまで見える
真面目なハートの傷痕
二人埋もれたね、やさしさ世界中
夢も消えるなら、いつか映像が見えるよ
行かない、海は終わらないよ
忘れず、懐かしい空に
時計は刻むよ、僕と誰かを
もうよそう、今日の君の笑顔
忘れずにいてよ
こんな力でキラキラ
込めた、繰り返し、繰り返し
＊
涙流れていく、やりきれない思いが重なり
「繰り返し」とずっと、愛は必ず届いていたよ
サヨナラの涙、とにかく僕が見えている
どれだけ心が、大人になるのか目覚めていよう
僕が手にしている体が、風を感じる
見慣れない世界へ続くよ、スタートラインを超えたら
時計を止めてよ、僕と誰かの
進み続ける時が来るから、遠くまで行くよ
言葉の退屈なんか、今も全部ここにはないよ
'''
  api_result  = cotoha_api.callSummaryApi(sentence, 1)
  print('結果 ＝ {}'.format(api_result['result']) )
  # 作詞は、ShikakiというAI作詞サポートツールを使用しました。

■ 構文解析
{'chunk_info': {'id': 0, 'head': 2, 'dep': 'D', 'chunk_head': 0, 'chunk_func': 0, 'links': []}, 'tokens': [{'id': 0, 'form': 'そう', 'kana': 'ソウ', 'lemma': 'そう', 'pos': '連用詞', 'features': [], 'dependency_labels': [], 'attributes': {}}]}
{'chunk_info': {'id': 1, 'head': 2, 'dep': 'D', 'chunk_head': 0, 'chunk_func': 1, 'links': []}, 'tokens': [{'id': 1, 'form': '今', 'kana': 'イマ', 'lemma': '今', 'pos': '名詞', 'features': ['日時', '連用'], 'dependency_labels': [{'token_id': 2, 'label': 'case'}], 'attributes': {}}, {'id': 2, 'form': 'は', 'kana': 'ハ', 'lemma': 'は', 'pos': '連用助詞', 'features': [], 'attributes': {}}]}
{'chunk_info': {'id': 2, 'head': 5, 'dep': 'D', 'chunk_head': 0, 'chunk_func': 1, 'links': [{'link': 0, 'label': 'manner'}, {'link': 1, 'label': 'time'}], 'predicate': []}, 'tokens': [{'id': 3, 'form': '冷た', 'kana': 'ツメタ', 'lemma': '冷たい', 'pos': '形容詞語幹', 'features': ['アウオ段'], 'dependency_labels': [{'token_id': 0, 'label': 'advmod'}, {'token_id': 1, 'label': 'nmod'}, {'token_id': 4,